## Select runs based on scaling restrictions and assumptions in AH2010

In this notebook I select the runs that match the dynamical restrictions imposed in \\Allen and Hickey, 2010\\ (AH2010). 

These restictions are:

1) Nearly uniform incoming flow along the length of the canyon.

2) Weak to moderate flow: $\mathcal{F}R_w<0.2$, where $\mathcal{F}=R_o/(0.9+R_o)$, $R_o=U/f\cal{R}$, $R_w=U/fW_s
$, $W_{s}$ is the mid-length width at shelf-break isobath.

3) Uniform stratification over the depth of upwelling.

4) Shallow shelf break $B_s=NH_s/fL<2$

5) Steep walls and stratified water column so that the bottom boundary layer is arrested within the canyon. See notebook 
EkmanLayerArrestTime.ipynb.

7) Narrow canyon: $W<2a$ where $a=NH_s/f$ is the Rossby raadius of deformation. (W at mid-length, rim depth)

8) Regular shape of the canyon at the upstream corner.

9) Sloped continental shelf.

Restrictions 5, 8 and 9 are geometric and satisified by all runs. Restriction 1 is also satisfied in all runs since they are forced in the same way. Restriction 3 is staisfied by all runs too since the initial stratification is uniform throughout the water column.

In the following, we select the runs that satisfy restrictions 2, 4 and 7.


In [1]:
import cmocean as cmo
import matplotlib.pyplot as plt
%matplotlib inline
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
import os
import sys

/home/kramosmu/anaconda3/lib/python3.5/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
# Set appearance options seaborn

sns.set_style('white')
sns.set_context('notebook')

In [3]:
lib_path = os.path.abspath('../PythonScripts/Paper1Figures') # Add absolute path to my python scripts
sys.path.append(lib_path)

import canyon_records 

records = canyon_records.main()

### Define constants and non dimensional numbers

In [4]:
# Constants and scales

g = 9.81 # accel. gravity
Hs = 149.8 # Shelf break depth
s = 0.01 # shelf slope 

# NOTE: The default values of all functions correspond to the base case

def Dh(f,L,N):
    '''Vertical scale Dh'''
    retun((f*L)/(N))
    
def Z(f, U, R, L, N):
    '''
    INPUT
    U: float: incomming velocity
    R: float: Radius of curvature shelf-break isobath
    f: float: Coriolis Parameter
    L: float: Canyon length
    N: float: initial stratification
    RETURNS
    Depth of upwelling from Allen and Hickey, 2010.
    '''
    return(((f*U*F(Ro(U,f,R))*L)**(0.5))/N)

def Ro(U,f,R):
    '''Rossby number using radius of curvature as length scale'''
    return(U/(f*R))

def F(Ro):
    '''Function that estimates the hability of the flow to follow isobaths'''
    return(Ro/(0.9+Ro))

def Bu(N,f,W,Hs):
    '''Burger number'''
    return((N*Hs)/(f*W))

def RossbyRad(N,Hs,f):
    '''1st Rossby radius of deformation'''
    return((N*Hs)/f)


In [11]:
for run in records:
    print(run.name)

CNTDIFF_run38
CNTDIFF_run37
CNTDIFF_run36
CNTDIFF_run45
CNTDIFF_run73
CNTDIFF_run75
CNTDIFF_run79
CNTDIFF_run67
CNTDIFF_run51
CNTDIFF_run69
CNTDIFF_run71
3DVISC_run01
3DVISC_run02
3DVISC_run04
3DVISC_run03
3DVISC_run05
3DVISC_run06
LOWER_BF_run01
LOW_BF_run01
LOWEST_BF_run01
LOWEST_BF_run03
LOWEST_BF_run05
LOWEST_BF_run07
LOWEST_BF_run11
3DVISC_REALISTIC_run01
3DVISC_REALISTIC_run02
3DVISC_REALISTIC_run03
3DVISC_REALISTIC_run05
3DVISC_REALISTIC_run06
3DVISC_REALISTIC_run07
3DVISC_REALISTIC_run08
3DVISC_REALISTIC_run09
3DVISC_REALISTIC_run10
3DVISC_REALISTIC_run11
3DVISC_REALISTIC_run12
3DVISC_REALISTIC_run13
3DVISC_REALISTIC_run14
3DVISC_REALISTIC_run15
3DVISC_REALISTIC_run16
3DVISC_REALISTIC_run17
3DVISC_REALISTIC_run18
3DVISC_REALISTIC_run19
3DVISC_REALISTIC_run20
3DVISC_REALISTIC_run21
3DVISC_REALISTIC_run22
3DVISC_REALISTIC_run23
3DVISC_REALISTIC_run24
3DVISC_REALISTIC_run25
3DVISC_REALISTIC_run26
3DVISC_REALISTIC_run27
3DVISC_REALISTIC_run28
CNTDIFF_run81
LONGER_CNY_run01
LONGER_C

### Check restrictions

In [17]:
for run in records[:]:
    
    # restriction 2, moderate flow: F*Rw < 0.2
    run.moderate = F(Ro(run.u_mod,run.f,run.R))*Ro(run.u_mod,run.f,run.Wiso)
    
    if run.moderate < 0.2:
        print('\033[1;31;32mrun %s satisfies moderate flow restriction: F*Rw = %f < 0.2' %(run.name,run.moderate))
    else:
        print('\033[1;31;30mrun %s doesn\'t satisfy moderate flow restriction: F*Rw = %f' %(run.name,run.moderate))
    
    # restriction 4, shallow shelf break: Bs<2
    run.Burger = Bu(run.N,run.f,run.L,Hs)
    
    if run.Burger <= 2:
        print('\033[1;31;32mrun %s satisfies shallow shelf restriction: Bu = %f <= 2' %(run.name,run.Burger))
    else:
        print('\033[1;31;30mrun %s doesn\'t satisfy shallow shelf restriction: Bu = %f' %(run.name,run.Burger))
    
    # restriction 7, narrow canyon: W < 2a
    run.narrow = 2*RossbyRad(run.N,Hs,run.f)
    
    if run.narrow >= run.W:
        print('\033[1;31;32mrun %s satisfies narrow canyon restriction: 2a = %f >= %f m \n' %(run.name,run.narrow, run.W))
    else:
        print('\033[1;31;30mrun %s doesn\'t satisfy narrow canyon restriction: 2a = %f < %f m \n' %(run.name,run.narrow, run.W))
    
    
    # Print Ro, Rl, Rw
    RO = Ro(run.u_mod,run.f,run.R)
    RL = Ro(run.u_mod,run.f,run.L)
    RW = Ro(run.u_mod,run.f,run.Wiso)
    print('\033[1;31;32mrun %s Rossby numbers are : Ro=%f, RL=%f, RWsb=%f  \n' %(run.name,RO,RL,RW))
    


run CNTDIFF_run38 satisfies moderate flow restriction: F*Rw = 0.132869 < 0.2
run CNTDIFF_run38 satisfies shallow shelf restriction: Bu = 1.023568 <= 2
run CNTDIFF_run38 doesn't satisfy narrow canyon restriction: 2a = 17057.971014 < 21141.700000 m 

run CNTDIFF_run38 Rossby numbers are : Ro=0.686994, RL=0.453456, RWsb=0.306935  

run CNTDIFF_run37 satisfies moderate flow restriction: F*Rw = 0.131161 < 0.2
run CNTDIFF_run37 satisfies shallow shelf restriction: Bu = 1.023568 <= 2
run CNTDIFF_run37 doesn't satisfy narrow canyon restriction: 2a = 17057.971014 < 21141.700000 m 

run CNTDIFF_run37 Rossby numbers are : Ro=0.681348, RL=0.449729, RWsb=0.304413  

run CNTDIFF_run36 satisfies moderate flow restriction: F*Rw = 0.107884 < 0.2
run CNTDIFF_run36 satisfies shallow shelf restriction: Bu = 1.023568 <= 2
run CNTDIFF_run36 doesn't satisfy narrow canyon restriction: 2a = 17057.971014 < 21141.700000 m 

run CNTDIFF_run36 Rossby numbers are : Ro=0.602296, RL=0.397551, RWsb=0.269094  

run CNT

In [18]:
for run in records:
    print(run.name, run.N, run.f)

CNTDIFF_run38 0.0055 9.66e-05
CNTDIFF_run37 0.0055 9.66e-05
CNTDIFF_run36 0.0055 9.66e-05
CNTDIFF_run45 0.0063 9.66e-05
CNTDIFF_run73 0.0074 9.66e-05
CNTDIFF_run75 0.0046 9.66e-05
CNTDIFF_run79 0.005 9.66e-05
CNTDIFF_run67 0.0055 0.0001
CNTDIFF_run51 0.0055 7.68e-05
CNTDIFF_run69 0.0055 8.6e-05
CNTDIFF_run71 0.0055 6.4e-05
3DVISC_run01 0.0055 9.66e-05
3DVISC_run02 0.0055 9.66e-05
3DVISC_run04 0.0055 9.66e-05
3DVISC_run03 0.0055 9.66e-05
3DVISC_run05 0.0055 9.66e-05
3DVISC_run06 0.0055 9.66e-05
LOWER_BF_run01 0.0055 9.66e-05
LOW_BF_run01 0.0055 9.66e-05
LOWEST_BF_run01 0.0055 9.66e-05
LOWEST_BF_run03 0.0046 9.66e-05
LOWEST_BF_run05 0.0074 9.66e-05
LOWEST_BF_run07 0.0055 7e-05
LOWEST_BF_run11 0.0055 9.66e-05
3DVISC_REALISTIC_run01 0.0055 9.66e-05
3DVISC_REALISTIC_run02 0.0055 9.66e-05
3DVISC_REALISTIC_run03 0.0055 9.66e-05
3DVISC_REALISTIC_run05 0.0055 9.66e-05
3DVISC_REALISTIC_run06 0.0055 9.66e-05
3DVISC_REALISTIC_run07 0.0055 9.66e-05
3DVISC_REALISTIC_run08 0.0055 9.66e-05
3DVISC_REAL

## Made-up runs

In [14]:
uu = 0.13
NN = 0.005
ff = 9.66E-5
Wiso = 
Wsb = 
W = 
L = 
R = 


# restriction 2, moderate flow: F*Rw < 0.2
moderate = F(Ro(uu,ff,R))*Ro(uu,ff,)
    
if moderate < 0.2:
    print('\033[1;31;32mrun satisfies moderate flow restriction: F*Rw = %f < 0.2' %(moderate))
else:
    print('\033[1;31;30mrun doesn\'t satisfy moderate flow restriction: F*Rw = %f' %(moderate))
    
# restriction 4, shallow shelf break: Bs<2
Burger = Bu(NN,ff,12800,Hs)
    
if Burger <= 2:
    print('\033[1;31;32mrun satisfies shallow shelf restriction: Bu = %f <= 2' %(Burger))
else:
    print('\033[1;31;30mrun doesn\'t satisfy shallow shelf restriction: Bu = %f' %(Burger))
    
# restriction 7, narrow canyon: Wsb < 2a
Narrow = 2*RossbyRad(NN,Hs,ff)
    
if Narrow >= Ws:
    print('\033[1;31;32mrun satisfies narrow canyon restriction: 2a = %f >= %f m \n' %(Narrow, Ws))
else:
    print('\033[1;31;30mrun doesn\'t satisfy narrow canyon restriction: 2a = %f < %f m \n' %(Narrow, Ws))
    
    
# Print Ro, Rl, Rw
RO = Ro(uu,ff,R)
RL = Ro(uu,ff,12800)
RWsb = Ro(uu,ff,Ws)
print('\033[1;31;32mrun Rossby numbers are : Ro=%f, RL=%f, RWsb=%f  \n' %(RO,RL,RWsb))
    


NameError: name 'R' is not defined